In [1]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
import scipy.io
import os
import glob
import pandas as pd
from scipy import signal
from hrv.classical import time_domain

import warnings
warnings.filterwarnings('ignore')

In [2]:
# トレーニングデータの格納先
path_training = './physionet.org/files/challenge-2017/1.0.0/training/'
# トレーニングデータの分割フォルダリスト
list_training_data_folder = ['A00','A01','A02']


# バンドパスフイルターのパラメータ
# 遮断周波数の最小値と最大値
# 心電図は一般的に0.05~500Hzの信号
cutoff_frequency = [0.05, 100]
# 今回のデータのサンプリング周波数は300
fs = 300
#　バンドパスフィルターの長さ、今回は最大100Hzの信号のため51
numtaps = 51


# カウント用の変数
i = 0

In [3]:
# 直流成分の除去
def RemoveDc(df):
    """
    パラメータ
    df : MATLAB形式のファイル読み取られた生の心電図波形
    
    return
    df : 心電図は非常に小さな電位であり、測定のために直流電流を付加することで、
    　　　信号を増幅しているので、これを除去したものを返す"""
    df['detrend'] = signal.detrend(df)
    
    return df

In [4]:
# バンドパスフィルター
def bandpass_ECG(df):
    """
    パラメータ
    df : df['detrend']のデータに対して0.05~100Hzのバンドパスフィルターを
    かけることで、心電図以外の信号できるだけ排除する
    
    変数
    filter_bandpass : バンドパスフィルターを設計している
    
    return
    df : バンドパスフィルターがかけられたコラムを追加したデータフレーム
    """
    
    # バンドパスフィルターの設計
    filter_bandpass = signal.firwin(numtaps=numtaps, cutoff=cutoff_frequency,
                                   fs=fs, pass_zero=False)
    # フィルターの適用
    df['bandpass'] = signal.lfilter(filter_bandpass,1,
                                           df['detrend'])
    
    return df

In [5]:
# 移動平均
def moving_average(df):
    """
    心電図波形には体動など筋電図や雑音によるスパイク（不自然に高電位成分が混じるもの）が
    含まれているので、移動平均処理をすることで軽減する
    移動平均が細かいとスパイクは軽減するが、細かすぎると胸部誘導心電図のような
    低電位の心電図のQRS（心室の興奮：ドキドキと拍動するときの信号）がかき消され、
    正確な心拍間隔（RRI）が測定できなくなるので、適度な細かさにする
    今回は17
    
    パラメータ
    df : バンドパスフィルターがかけられたデータ
    
    return
    df : 移動平均後のデータを追加
    """
    df['moving_average'] = df['bandpass'].rolling(17).mean()
    
    return df

In [6]:
def find_peak_RRI(df,high_level=6e-13,low_level=0.2e-13):
    """
    HRV(心拍変動解析)を行うために心拍間隔（RRI）の検出を行う
    RRIは心拍と次の心拍までの時間のことで、これを解析することで不整脈を検出することができる
    今回は心房細動の検出
    
    パラメータ
    df : 移動平均後のデータに対してRRIを検出する
    
    return
    df_rri : ms単位のrri信号を返す"""

    # R波ピークを検出するときの最大電位と最小電位のしきい値
    border_high = np.full(len(df['moving_average']), high_level)
    border_low = np.full(len(df['moving_average']), low_level)
    # RRIの検出、インデックスが返されてくる
    peaks_index, _ = signal.find_peaks(df['moving_average'], distance=120, height=(border_low,border_high))
    # インデックスの差分を取ることでひとつのRRIごとの心拍間隔がわかる
    peaks_index_diff = np.diff(peaks_index)
    # 今回の心電図はサンプリング周波数300Hz、つまり、1秒間に300回記録している
    # 各値を300で割るとRRI[sec]が取得できる
    rri_sec = peaks_index_diff / 300
    # 単位をmsにするために1000倍する
    rri_ms = rri_sec * 1000
    # これをDataFrameに変換する
    df_rri = pd.DataFrame(rri_ms, columns=['rri'])
    # 時系列データの頻度をmsにしておく
    df_rri.index.freq = 'ms'
    
    return df_rri


In [7]:
# 瞬時心拍を追加する
def heart_rate(df):
    """
    (60 / RRI) * 1000よりその瞬間の心拍数がわかるのでその情報を追加する
    パラメータ
    df : RRIデータに対してに上記の演算を行い心拍数を求める
    
    return
    df : 心拍データを追加したデータフレーム
    """
    df['HR'] = (60 / df['rri']) * 1000
    
    return df

In [8]:
# HRV解析を行う
def hrv_analysis(df, file_path):
    """
    ライブラリ hrvのtime_domainを使用することで下記のデータが取得できる
    rmssd : 隣接するRRIの差の平均の２乗根
    　　　　  心房細動はRRIが不規則なのでこの値は大きくなりやすい
    sdnn : データ全体のRRIの標準偏差
           心筋梗塞などではsdnnは小さく100ms以上ならば問題ないが、
           心房細動は心拍数が100拍以上になるような頻脈性と
           心拍数が50以下になるような徐脈性があるので大きすぎても小さすぎてもいけない
           心房細動のRRIの不規則さを反映する
    ln(sdnn) : 測定時間が短いデータではsdnnを自然対数で処理すると良いという
               先行研究より追加
    sdsd : 隣接するrriの標準偏差
           これもRRIの不規則さを反映
    nn50 : ある時点のRRIと次のRRIの差が50ms以上になる区間が何回あるかを示す
           正常心電図でもRRIは完璧に一定ではないので多少はnn50が検出される
           しかし、心房細動ではこの回数が大きくなることもある
           nn50が多い場合は徐脈性の場合が考えられる
    pnn50 : nn50が測定データに占める割合
            心房細動では大きくなりやすいこともある
    mrri : 平均RRI
           RRIが600ms以下なら頻脈性、1200ms以上なら徐脈性
    mhr : 平均心拍数
          心拍数の正常範囲は60以上100以下
    stdhr : 心拍数の標準偏差
            hrvライブラリには含まれてないが、臨床的な特徴量として注目したいので追加
    CVRR : (SDNN/平均RRI) * 100より算出する
           統計学の変動係数のことであり、平均RRIあたりどの程度ばらつきがあるのかわかる
           心房細動なら大きな値になると考えられる(これも追加項目)
    
    パラメータ
    df : RRIと瞬時心拍データに対して処理を行う
    file_path : 読み込み中のファイルパス
    
    return
    df_hrv : HRV解析データを格納した横長のデータフレーム
    """
    if len(df) == 0:
        print(file_path)

    # HRV解析、辞書が返ってくる
    hrv_rri_dict = time_domain(df.iloc[:,0].values)
    # 読み込み中のデータpathからファイルネームを取得
    file_name_index = file_path[-10:-4].split(maxsplit=0)
    # HRVデータの入った辞書をデータフレームに変換
    df_hrv = pd.DataFrame(hrv_rri_dict.values(), index=hrv_rri_dict.keys(),
                              columns=file_name_index)
    # 転置
    df_hrv = df_hrv.T
    # 自然対数でsdnnを処理
    df_hrv['ln(sdnn)'] = np.log(df_hrv['sdnn'].values)
    # 心拍標準偏差を追加
    df_hrv['stdhr'] = df['HR'].std()
    # CVRRを追加
    df_hrv['cvrr'] = (df_hrv['sdnn'] / df_hrv['mrri']) * 100
    
    return df_hrv

In [9]:
# メインの処理:学習データを一つのデータフレームに加工
def main(i=i):
 
    # １ファイルごとのデータフレームを格納するデータフレーム
    df_trainigset_onefile = pd.DataFrame(index=[],columns=[])
    # 別々のフォルダに保存されているファイルパスを取得していくループ
    for training_folder in list_training_data_folder:

        path_traingset = glob.glob(path_training+training_folder)


        for path_traingset_local in path_traingset:
            
            path_training_file = glob.glob(path_traingset_local + '/*.mat')

            
            # 一つずつファイルに処理を行う
            for path_training_file_local in path_training_file:
                
            
                # .mat形式のファイルを読み込み
                mat_file = scipy.io.loadmat(path_training_file_local)
                # リスト変換
                mat_file_list_ecg = list(mat_file.values())
                # 生波形データフレーム
                df_pure = pd.DataFrame(np.ravel(mat_file_list_ecg),columns=['pure'])

                df_detrend = RemoveDc(df_pure)
                
                df_bandpass = bandpass_ECG(df_detrend)
                
                df_moving_average = moving_average(df_bandpass)
                
                df_find_peaks = find_peak_RRI(df_moving_average)

                # 移動平均後が低電位すぎてRRIが検出できない場合は検出値の下限を引き下げる
                if len(df_find_peaks) == 0:
                    df_find_peaks = find_peak_RRI(df=df_moving_average,low_level=0.1e-13)

                df_HR = heart_rate(df_find_peaks)
                
                path_data_str = ''.join(path_training_file_local)
                df_hrv = hrv_analysis(df_HR,path_data_str)
                
                df_trainigset_onefile = pd.concat([df_trainigset_onefile, df_hrv],axis=0)
                i =i + 1

#                

#             display(df_trainigset_onefile)
            print(f'フォルダ : {path_traingset_local[-3:]}')
            print(f'ファイル数 : {i}')
            print('============')
            
    return df_trainigset_onefile

df =  main()

フォルダ : A00
ファイル数 : 999
フォルダ : A01
ファイル数 : 1999
フォルダ : A02
ファイル数 : 2854


In [10]:
# indexを昇順にして整理
df = df.sort_index(ascending=True)
display(df)

,rmssd,sdnn,sdsd,nn50,pnn50,mrri,mhr,ln(sdnn),stdhr,cvrr
A00001,100.643575,65.318577,102.027740,4.0,10.526316,761.666667,79.330866,4.179276,6.890820,8.575743
A00002,290.780200,222.416471,295.066494,19.0,57.575758,893.434343,72.730171,5.404552,24.004782,24.894551
A00003,229.549405,155.065837,231.053020,32.0,41.025641,763.760684,82.196182,5.043850,19.738613,20.302935
A00004,237.096636,189.473773,241.103148,24.0,77.419355,936.021505,66.654705,5.244251,13.312296,20.242459
A00005,292.018724,211.556696,293.689064,64.0,73.563218,668.084291,98.951104,5.354493,30.106937,31.666168
...,...,...,...,...,...,...,...,...,...,...
A02850,501.102616,337.447882,509.972042,22.0,73.333333,975.000000,69.394540,5.821411,27.193320,34.610039
A02851,371.104004,229.615228,376.855538,18.0,52.941176,744.509804,86.569177,5.436405,22.931642,30.841129
A02852,477.962646,311.771887,486.788060,20.0,74.074074,986.666667,68.833104,5.742272,28.728158,31.598502
A02853,365.724726,269.362321,371.066149,31.0,88.571429,851.047619,78.090732,5.596057,26.313719,31.650676


In [11]:
# Nanが含まれたデータについてはラベル付け用のnotebookで
# ラベル付後に欠損値が含まれるサンプルを除去する
df.isnull().sum(axis=0)

rmssd       0
sdnn        0
sdsd        1
nn50        0
pnn50       0
mrri        0
mhr         0
ln(sdnn)    0
stdhr       0
cvrr        0
dtype: int64

In [12]:
# データフレームの保存
os.mkdir('./training_DataFrame')
save_path_dir = './training_DataFrame'

def dataframe_save(df, save_path_dir, file_name):
    """
    CSVとpickle形式で保存
    pickleは読み書きが高速
    パラメータ
    df : 保存したいデータフレーム
    save_path_dir : 保存先フォルダのパス
    file_name : 保存したいファイルネーム
    """
    # 保存先パス
    path_save = os.path.join(save_path_dir + '/' + file_name)
    
    # CSV形式で保存
    df.to_csv(path_save + '.csv', index=True, header=True, sep=',')
    # pickle形式で保存
    df.to_pickle(path_save + '.pkl')

dataframe_save(df=df, save_path_dir=save_path_dir, file_name='training_dataset_non_label')